In [1]:
import os
print(os.getcwd())
os.chdir("../")
os.getcwd()

/home/mobo/Documents/Ineuron/Live_Class/DL/DL_live_notes/CNN/deepCNNClassifier/research


'/home/mobo/Documents/Ineuron/Live_Class/DL/DL_live_notes/CNN/deepCNNClassifier'

In [2]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list
    
@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [3]:
from deepClassifier.config import ConfigurationManager
from deepClassifier.utils import read_yaml, create_directories
from deepClassifier.constants import (
    CONFIG_FILE_PATH,
    PARAMS_FILE_PATH
)


class ConfigurationManager:
    
    def __init__(self,
                 config_filepath: Path=CONFIG_FILE_PATH,
                 parmas_filepath: Path=PARAMS_FILE_PATH
                 ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(parmas_filepath)
        
        create_directories([self.config.artifacts_root])

    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config
           
    def get_training_config(self) -> TrainingConfig:
        config = self.config.training
        prepare_base_model = self.config.prepare_base_model
        data_ingestion = self.config.data_ingestion
        training_data = os.path.join(
            self.config.data_ingestion.unzipped_dir, "PetImages")
        
        create_directories([config.root_dir])
        
        return TrainingConfig(
            root_dir=config.root_dir,
            trained_model_path=config.trained_model_path,
            updated_base_model_path=prepare_base_model.updated_base_model_path,
        training_data = training_data,
            params_epochs=self.params.EPOCHS,
            params_batch_size=self.params.BATCH_SIZE,
            params_is_augmentation=self.params.AUGUMENTATION,
            params_image_size=self.params.IMAGE_SIZE
        )

[2022-09-23 20:41:24,532: INFO: common]: yaml file: configs/config.yaml loaded successfully
[2022-09-23 20:41:24,602: INFO: common]: yaml file: params.yaml loaded successfully
[2022-09-23 20:41:24,604: INFO: common]: created directory at: artifacts
[2022-09-23 20:41:24,606: INFO: common]: created directory at: artifacts/prepare_callbacks/tensorboard_log_dir
[2022-09-23 20:41:24,609: INFO: common]: created directory at: artifacts/prepare_callbacks/checkpoint_dir


In [4]:
config = ConfigurationManager()
config.get_training_config()

[2022-09-23 20:41:37,449: INFO: common]: yaml file: configs/config.yaml loaded successfully
[2022-09-23 20:41:37,453: INFO: common]: yaml file: params.yaml loaded successfully
[2022-09-23 20:41:37,455: INFO: common]: created directory at: artifacts
[2022-09-23 20:41:37,457: INFO: common]: created directory at: artifacts/training


TrainingConfig(root_dir='artifacts/training', trained_model_path='artifacts/training/model.h5', updated_base_model_path='artifacts/prepare_base_model/base_model_updated.h5', training_data='artifacts/data_ingestion/PetImages', params_epochs=1, params_batch_size=16, params_is_augmentation=True, params_image_size=BoxList([224, 224, 3]))

In [6]:
import tensorflow as tf


import time

class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config

    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )

    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]



In [7]:
import tensorflow as tf


class Training:
    
    def __init__(
        self,
        training_config: TrainingConfig
    ):
        self.config = training_config
        
    def get_base_model(self) -> None:
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
        
    def train_valid_generator(self):
        # goal: split training data in train and validation set and pass its params
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )
        
        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )
        
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )
        
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )
        
        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=True,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator
        
        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )
        
        
    def train(self, callback_list: list) -> None:
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size
        
        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=callback_list
        )
        
        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )
        
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model) -> None:
        model.save(path)
        
        

In [8]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()
    
    training_config = config.get_training_config()
    training = Training(training_config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(
        callback_list=callback_list
    )
    
except Exception as e:
    raise e

[2022-09-23 20:42:16,818: INFO: common]: yaml file: configs/config.yaml loaded successfully
[2022-09-23 20:42:16,822: INFO: common]: yaml file: params.yaml loaded successfully
[2022-09-23 20:42:16,824: INFO: common]: created directory at: artifacts
[2022-09-23 20:42:16,826: INFO: common]: created directory at: artifacts/prepare_callbacks/checkpoint_dir
[2022-09-23 20:42:16,829: INFO: common]: created directory at: artifacts/prepare_callbacks/tensorboard_log_dir
[2022-09-23 20:42:16,832: INFO: common]: created directory at: artifacts/training


2022-09-23 20:42:17.134431: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-09-23 20:42:17.134475: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-09-23 20:42:17.134498: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (mobo): /proc/driver/nvidia/version does not exist


Found 4998 images belonging to 2 classes.
Found 20000 images belonging to 2 classes.


2022-09-23 20:42:24.031182: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 205520896 exceeds 10% of free system memory.
2022-09-23 20:42:25.077752: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 205520896 exceeds 10% of free system memory.


   1/1250 [..............................] - ETA: 5:02:55 - loss: 0.7148 - accuracy: 0.4375

...

   2/1250 [..............................] - ETA: 4:41:27 - loss: 0.7224 - accuracy: 0.5312

2022-09-23 20:42:50.118002: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 205520896 exceeds 10% of free system memory.


1018/1250 [=======================>......] - ETA: 2:49:19 - loss: 5.7534 - accuracy: 0.7071

/home/mobo/Documents/Ineuron/Live_Class/DL/DL_live_notes/CNN/deepCNNClassifier/env/lib/python3.8/site-packages/PIL/TiffImagePlugin.py:845: UserWarning: Truncated File Read
  warnings.warn(str(msg))


1250/1250 [==============================] - 51526s 41s/step - loss: 5.2509 - accuracy: 0.7212 - val_loss: 1.1785 - val_accuracy: 0.8926
